# Acquiring and Processing Information on the World's Largest Banks

In this project, we will work with real-world data and perform the operations of Extraction, Transformation, and Loading (ETL) as required.

* Task 1: Logging function
* Task 2: Extraction of data
* Task 3: Transformation of data
* Task 4: Loading to CSV
* Task 5: Loading to Database
* Task 6: Function to Run queries on Database
* Task 7: Verify log entries

### Preliminaries: Installing libraries and downloading data

Install the required libraries

In [1]:
!pip install numpy pandas bs4 wget

  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached wget-3.2-py3-none-any.whl
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)


## Importing Libraries
Importing the required libraries

In [11]:
import requests
import sqlite3
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

# Suppress generated warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

## Tasks
Task 1: Logging function
This function accepts the message to be logged and enters it to the log file on a new line.

Log entries will be in the format: <timestamp> : <message>

In [40]:
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''

    timestamp_format = "%Y-%h-%d-%H:%M:%S" # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open(log_file,"a") as f:
        f.write(timestamp + " : " + message + "\n")

Task 2: Extraction of data

Inspect the URL and identify the position and pattern of the tabular information in the HTML code under the heading By market capitalization. In the given webpage, our table is at the first position, or index 0 and we require the entries under "Bank name" and "Market cap (US$ billion)".



Write the function extract() to retrieve the information of the table to a Pandas DataFrame

In [1]:
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''

    page = requests.get(url).text
    soup = BeautifulSoup(page, "html.parser")

    df = pd.DataFrame(columns=table_attribs)

    tables = soup.find_all("tbody")
    rows = tables[0].find_all("tr")

    for row in rows:
        col = row.find_all("td")
        if len(col) != 0:
            data_dict = {"Name": col[1].find_all("a")[1]["title"],
                         "MC_USD_Billion": float(col[2].contents[0][:-1])}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df, df1], ignore_index=True)

    return df


The code functions as follows:

Extract the web page as text.
Parse the text into an HTML object.
Create an empty Pandas DataFrame named df with columns argument set as table_attribs.
Extract all tbody attributes of the HTML object and then extract all the rows of the index 0 table using the tr attribute.
Iterate over the contents of the variable rows.
Extract all the td data objects in the row and save them to col.
Check if the length of col is 0, that is, if there is no data in the current row. This is important since, many times there are merged rows that are not apparent in the web page appearance.
Create a dictionary data_dict with the keys same as the columns of the dataframe created for recording the output earlier and corresponding values from the second and third headers of data.
Extract the title attribute of the second hyperlink from the Bank name column.
Remove the last character (\n) from the Market cap (US$ billion) column contents using negative index slicing, then typecast the value to float format.
Convert the dictionary to a dataframe and concatenate it with the existing one. This way, the data keeps getting appended to the dataframe with every iteration of the loop.
Return the dataframe.

## Task 3: Transformation of data
The Transform function needs to perform the following tasks:

Read the exchange rate CSV file and convert the contents to a dictionary so that the contents of the first columns are the keys to the dictionary and the contents of the second column are the corresponding values.

Add 3 different columns to the dataframe: MC_GBP_Billion, MC_EUR_Billion and MC_INR_Billion, each containing the content of MC_USD_Billion scaled by the corresponding exchange rate factor. Round the resulting data to 2 decimal places.

In [2]:
def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''

    # Read exchange rate CSV file
    exchange_rate = pd.read_csv(csv_path)

    # Convert to a dictionary with "Currency" as keys and "Rate" as values
    exchange_rate = exchange_rate.set_index("Currency").to_dict()["Rate"]

    # Add MC_GBP_Billion, MC_EUR_Billion, and MC_INR_Billion
    # columns to dataframe. Round off to two decimals
    df["MC_GBP_Billion"] = [np.round(x * exchange_rate["GBP"], 2) for x in df["MC_USD_Billion"]]
    df["MC_EUR_Billion"] = [np.round(x * exchange_rate["EUR"], 2) for x in df["MC_USD_Billion"]]
    df["MC_INR_Billion"] = [np.round(x * exchange_rate["INR"], 2) for x in df["MC_USD_Billion"]]

    return df

### Task 4: Loading to CSV
Load the transformed dataframe to an output CSV file.

In [3]:
def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''

    df.to_csv(output_path)

### Task 5: Loading to Database
Load the transformed dataframe to a SQL database server as a table.

In [4]:
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''

    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

## Task 6: Function to Run queries on Database
Run queries on the database table.


In [5]:
def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''

    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

## Running ETL Process
Declaring known values

In [46]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
csv_path = "./exchange_rate.csv"
table_attribs = ["Name", "MC_USD_Billion"]
output_path = "./Largest_banks_data.csv"
db_name = "Banks.db"
table_name = "Largest_banks"
log_file = "./code_log.txt"

import logging

# Define the log_progress function
def log_progress(message):
    logging.basicConfig(filename=log_file, level=logging.INFO)
    logging.info(message)
    print(message)  # Optional: print the message to the console for visibility

# Call the function
log_progress("Preliminaries complete. Initiating ETL process")


Preliminaries complete. Initiating ETL process


In [12]:
df = extract(url, table_attribs)
print(df)

log_progress("Data extraction complete. Initiating Transformation process")

                                      Name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                                     HSBC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81
Data extraction complete. Initiating Transformation process


In [14]:
df = transform(df, csv_path)
print(df)

log_progress("Data transformation complete. Initiating Loading process")

                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                                     HSBC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75

In [16]:
import sqlite3
import pandas as pd

# Example DataFrame (df) and output path
df = pd.DataFrame({
    "Name": ["Bank A", "Bank B", "Bank C"],
    "MC_USD_Billion": [500, 450, 400]
})
output_path = "./Largest_banks_data.csv"
db_name = "Banks.db"
table_name = "Largest_banks"

# Function to save the DataFrame to CSV
def load_to_csv(df, output_path):
    df.to_csv(output_path, index=False)

# Save the data to a CSV file
load_to_csv(df, output_path)
log_progress("Data saved to CSV file")

# Initiate SQLite3 connection
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# Create a table in SQLite if it doesn't exist
cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS {table_name} (
        Name TEXT,
        MC_USD_Billion REAL
    )
''')

# Insert data into the table
for index, row in df.iterrows():
    cursor.execute(f'''
        INSERT INTO {table_name} (Name, MC_USD_Billion)
        VALUES (?, ?)
    ''', (row['Name'], row['MC_USD_Billion']))

# Commit the transaction and close the connection
conn.commit()
conn.close()

log_progress("Data loaded into SQLite3 database")


Data saved to CSV file
Data loaded into SQLite3 database


In [18]:
import sqlite3
import pandas as pd

# Assuming df is your DataFrame to be inserted into the database
df = pd.DataFrame({
    "Name": ["Bank A", "Bank B", "Bank C"],
    "MC_USD_Billion": [500, 450, 400]
})

# Path to the SQLite database
db_name = "Banks.db"
table_name = "Largest_banks"

# Function to load data into SQLite database
def load_to_db(df, db_name, table_name):
    '''This function loads data from the DataFrame into the SQLite database.'''
    
    # Establish SQLite connection
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # Create table if it doesn't exist
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {table_name} (
            Name TEXT,
            MC_USD_Billion REAL
        )
    ''')
    
    # Insert data into the table
    for index, row in df.iterrows():
        cursor.execute(f'''
            INSERT INTO {table_name} (Name, MC_USD_Billion)
            VALUES (?, ?)
        ''', (row['Name'], row['MC_USD_Billion']))
    
    # Commit and close the connection
    conn.commit()
    conn.close()

# Establish SQL connection
sql_connection = sqlite3.connect(db_name)
log_progress("SQL Connection initiated")  # Log the progress

# Call the function to load data into the database
load_to_db(df, db_name, table_name)  # No 'Call' keyword needed

log_progress("Data loaded into SQLite database")  # Log after loading the data


SQL Connection initiated
Data loaded into SQLite database


In [34]:
import sqlite3
import pandas as pd

# Example DataFrame (df)
df = pd.DataFrame({
    "Name": ["Bank A", "Bank B", "Bank C"],
    "MC_USD_Billion": [500, 450, 400]
})

# Path to the SQLite database
db_name = "Banks.db"
table_name = "Largest_banks"

# Function to load data into SQLite database (using the active connection)
def load_to_db(df, conn, table_name):
    '''This function loads data from the DataFrame into the SQLite database using an existing connection.'''
    cursor = conn.cursor()
    
    # Create table if it doesn't exist
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {table_name} (
            Name TEXT,
            MC_USD_Billion REAL
        )
    ''')
    
    # Insert data into the table
    for index, row in df.iterrows():
        cursor.execute(f'''
            INSERT INTO {table_name} (Name, MC_USD_Billion)
            VALUES (?, ?)
        ''', (row['Name'], row['MC_USD_Billion']))
    
    # Commit the transaction
    conn.commit()

# Establish SQL connection
sql_connection = sqlite3.connect(db_name)
log_progress("SQL Connection initiated")  # Log the progress

# Call the function to load data into the database (use the existing connection)
load_to_db(df, sql_connection, table_name)

log_progress("Data loaded to Database as a table, Executing queries")  # Log after loading the data


SQL Connection initiated
Data loaded to Database as a table, Executing queries


##1. Print the contents of the entire table

In [36]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def extract(url, table_attribs):
    # Send request to the URL and parse the content
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the table based on the given attributes
    table = soup.find('table', attrs=table_attribs)
    
    # Parse the table into a DataFrame
    df = pd.read_html(str(table))[0]  # Assuming there's only one table on the page
    return df


In [39]:
import pandas as pd

# Assuming extract and log_progress are defined elsewhere

# Extract data using the extract() function
df = extract(url, table_attribs)

# Print the extracted data to verify
print(df)

# Log progress
log_progress("Data extraction complete. Initiating Transformation process.")

# Define the transform() function to process the data
def transform(df):
    # Example transformation: Convert USD market cap to millions (instead of billions)
    df['MC_USD_Million'] = df['MC_USD_Billion'] * 1000  # Convert billions to millions
    
    # Example transformation: Add a column that indicates if market cap is over 200 billion
    df['High_Market_Cap'] = df['MC_USD_Billion'] > 200
    
    return df

# Call the transform function to process the extracted data
transformed_df = transform(df)

# Print the transformed data to verify
print(transformed_df)



                                      Name  MC_USD_Billion  MC_GBP_Billion  
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                                     HSBC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion       MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75  
3          149.43        13328.41  
4          146.86        13098.63  
5          144.96        12929.42  
6          138.48        12351.26  
7          130.97        11681.85  
8          130.03        11598.07  
9          127.23        11348.39  


IndentationError: unexpected indent (2086909656.py, line 32)

2. Print the average market capitalization of all the banks in Billion GBP

In [44]:
SELECT AVG(MC_GBP_Billion) FROM Largest_banks;


SyntaxError: invalid syntax (556639637.py, line 1)

3.Print only the names of the top 5 banks

In [47]:
query_statement = f"SELECT Name from {table_name} LIMIT 5"
run_query(query_statement, sql_connection)

log_progress("Process Complete")

SELECT Name from Largest_banks LIMIT 5
     Name
0  Bank A
1  Bank B
2  Bank C
3  Bank A
4  Bank B
Process Complete


## Task 7: Verify log entries
Upon successful completion of execution, you should see all the relevant entries made in the log file in relation to the stages of code execution.

In [48]:
with open(log_file, "r") as log:
    LogContent = log.read()
    print(LogContent)

INFO:root:Preliminaries complete. Initiating ETL process
INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:root:Data extraction complete. Initiating Transformation process
INFO:root:Data transformation complete. Initiating Loading process
INFO:root:Data saved to CSV file
INFO:root:Data loaded into SQLite3 database
INFO:root:SQL Connection initiated
INFO:root:Data loaded into SQLite database
INFO:root:SQL Connection initiated
INFO:root:Data loaded to Database as a table, Executing queries
INFO:root:SQL Connection initiated
INFO:root:Data loaded to Database as a table, Executing queries
INFO:root:Process Complete
INFO:root:Preliminaries complete. Initiating ETL process
INFO:root:Process Complete



## Christian Reyes
11/16/2024